### Knapsack Scratch

This is for Week 2 of the Discrete Optimization lecture. I use Jupyter notebooks for scratch and then move them into .py files and write tests in PyCharm. Keeping this as notes for my own optimization problems - both reference code, and questions I meant to explore but may not have.

**Open questions:**
* My runtimes appear significantly slower than the "leaderboard" for the project (often sub-second). 
    * What are some opportunities for improvement here?
    * How should I think about measuring performance of an OR solution and assessing scale?
* What is required to create guarantees for different optimization strategies?

### Dynamic Programming

The DP strategy is to create a running tracker of the optimal solution by introducing new items iteratively. We can then infer which items are chosen the optimal strategy by analyzing the output.


In [3]:
# imports
import pandas as pd
import os
from collections import namedtuple
import numpy as np

# get names of all data files

working_dir = os.path.abspath(os.path.join(os.getcwd(),".."))
datapath = 'data'
onlyfiles = [f for f in os.listdir(os.path.join(working_dir,datapath)) 
                     if os.path.isfile(os.path.join(working_dir,datapath,f))]

# choose smallest file as a test file
small_file = 'ks_4_0'
file_location = os.path.join(working_dir,datapath,small_file)

In [18]:
class dp:
    
    def __init__(self, input_data):
        self.input_data = input_data
    
    def dynamic_programming_algo(self):
        '''Generates a table for dynamic programming and infers output.
           Uses significant space and time O(k*n), but guaranteed optimal.'''
        
        # load the data
        items, item_count, capacity = self._load_data()

        # build the table
        dp_table = self._build_table(items, item_count, capacity)

        # generate output
        output_data = self._generate_output(dp_table, items, item_count, capacity)

        return output_data

    def _load_data(self):
        '''Takes Coursera input files splits them, and creates a tuple in 
           their preferred format. Unsorted.'''
        
        Item = namedtuple("Item", ['index', 'value', 'weight', 'density'])

        # parse the input
        lines = self.input_data.split('\n')

        first_line = lines[0].split()
        item_count = int(first_line[0])
        capacity = int(first_line[1])

        items = []

        for i in range(1, item_count+1):
            line = lines[i]
            parts = line.split()
            items.append(Item(i-1, int(parts[0]), int(parts[1]), float(parts[0])/float(parts[1])))

        return items, item_count, capacity

    @staticmethod
    def _build_table(items, item_count, capacity):
        '''Create a table that is the capacity of the knapsack+1 and the number of items+1.
           For every item added, calculate the optimal solution at every capacity. This is
           done by iteratively building the table, and referring back to the prior optimal
           solution (ie. from the last item's solution).'''
        
        # initialize the table
        dp_table = np.zeros(shape=(capacity + 1, item_count + 1))

        for j in range(item_count):
            item = items[j]
            for i in range(capacity + 1):
                current_weight = item.weight
                current_value = item.value

                value_if_include = 0
                prior_knapsack_at_weight = dp_table[i, j]
                
                # determine what the weight would be if the new item is included
                if current_weight <= i:
                    prior_knapsack_if_included = dp_table[max(i-current_weight, 0), j]
                    value_if_include = prior_knapsack_if_included + current_value

                if value_if_include > prior_knapsack_at_weight:
                    dp_table[i, j+1] = value_if_include
                else:
                    dp_table[i, j+1] = prior_knapsack_at_weight

        return dp_table

    @staticmethod
    def _generate_output(dp_table, items, item_count, capacity):
        '''With the table output, determine the items that compose the optimal solution.
           This can be determined by reverse engineering the table (eg, if the "maximum value"
           was the same for a given capacity for the last item, the current item was not added.'''
        
        # initialize
        items_taken = [0]*item_count
        current_item = item_count
        current_capacity = capacity
        prior_item = current_item - 1
        optimal_value = int(dp_table[current_capacity, current_item])

        for current_item in range(current_item, 0, -1):
            # compare if max_value is different, determine if current item is included
            if dp_table[current_capacity, current_item] != dp_table[current_capacity, prior_item]:
                items_taken[current_item-1] = 1
                
                # reset for next iteration
                current_capacity = current_capacity - items[current_item-1].weight
                current_item = prior_item
                prior_item = current_item - 1

            # current item is not included, move over one spot
            else:
                current_item = prior_item
                prior_item = current_item - 1

        # prepare the solution in the specified output format
        output_data = str(optimal_value) + ' ' + str(0) + '\n'
        output_data += ' '.join(map(str, items_taken))

        return output_data

In [19]:
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
check_output = dp(input_data=input_data)
check_output.dynamic_programming_algo()

'99798 0\n0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0'

## DFS, with branching and bounding

To improve runtime and space constraints, depth first search employs a branching and bounding strategy to remove some of the search area. The solution does not have guarantees. 

We are able to improve the bounding strategy by "relaxing" - calculating the optimal fractional value that a branch is capable of creating, and determining if that is an improvement over the current optimal soution. This is done by creating a "density metric" (value / weight) and then sorting the items using that metric.

In [8]:
# load imports
import pandas as pd
import os
from collections import namedtuple
import numpy as np

big_file = 'ks_30_0'
small_file = 'ks_4_0'
datapath = 'data'

file_location = os.path.join(working_dir,datapath,big_file)

In [20]:
class dfs:
    '''This class employs a depth first search strategy - the main function is explore_branch(),
       which will traverse a branch until it:
           (1) uses up too much weight (the "floor" is negative), 
           (2) is exhausted, or 
           (3) does not have the potential to have more value than the current best answer.
           
        The state of the next branch for exploring is calculated using the next_branch() function,
        which updates the next node for searching, as well as the starting floor and max potential
        value.'''
    
    def __init__(self, input_data):
        '''As the algo traverses different branches, many variables are required to keep state - 
           for global values as well as updating nodes to explore.'''
        
        # global variables
        self.input_data = input_data
        self.iterations = 0
        self.items, self.item_count, self.capacity = self._load_data()
        self.best_value = 0
        self.best_set = np.zeros(len(self.items)).astype(int)
        
        # node-specific variables - "kept_value" is value accrued, "floor" is weight left
        self.kept_value = 0
        self.floor = self.capacity
        
        self.current_set = np.zeros(len(self.items)).astype(int)
        self.current_level = 0
        self.max_potential_value = self._value_estimate(items=self.items, 
                                                        capacity=self.capacity, 
                                                        level=self.current_level, 
                                                        function_set=np.ones(len(self.items)).astype(int), 
                                                        iterations=self.iterations)
        self.current_max_value = self.max_potential_value
        self.next_level = self.current_level
    
    def explore_branch(self):
        '''Traverses an individual branch until it:
           (1) uses up too much weight (the "floor" is negative), 
           (2) is exhausted, or 
           (3) does not have the potential to have more value than the current best answer.
           
           Then updates state for next exploration. This function is used in a while loop.
           '''
        
        for item in range(self.next_level, len(self.items), 1):
            # if we already have a best value that is higher than what is possible, break
            if self.current_max_value < self.best_value:
                break

            # if the next item leads us to go "over" in weight, break otherwise, include 
            # and update state. If we are at the end of a branch, update
            if self.items[item].weight <= self.floor:
                self.kept_value += self.items[item].value
                self.floor -= self.items[item].weight
                self.current_set[item] = 1  

                if self.kept_value > self.best_value:
                    self.best_value = self.kept_value
                    self.best_set = self.current_set.copy()    

            # when we go over from weight, update state and break
            else:
                break
                
        self.iterations += 1
        
        return self.best_value, self.best_set, self.iterations, self.next_branch(item)
    
    def next_branch(self, item):
        '''Updates state values for next branch to be traversed. Calculates:
           (1) max_potential_value
           (2) floor
           (3) kept_value
           (4) index_level and item set'''
        
        # update floor and max_potential_value
        if item == len(self.items) - 1: # branch exhausted
            if self.current_set[0] == 1:  
            # find first 0, set level to one higher and that value to 0
                self.current_level = np.where(self.current_set == 0)[0][0] - 1                
                self.current_set[self.current_level] = 0
                self.current_set[item] = 1
                
            else:
                # find first 1, set value to 0 and make current level; subsequent levels to 1
                self.current_level = np.where(self.current_set == 1)[0][0]                
                self.current_set[self.current_level] = 0
                self.current_set[self.current_level+1:] = 1
        else:
            self.current_level = item
        
        # calculate floor and kept value
        self.floor, self.kept_value = self._update_floor_and_kept_value(items=self.items, 
                                                                        capacity=self.capacity, 
                                                                        level=self.current_level, 
                                                                        function_set=self.current_set.copy(),
                                                                        iterations=self.iterations)
        # calculate max value
        self.current_max_value = self._value_estimate(items=self.items, 
                                                        capacity=self.capacity, 
                                                        level=self.current_level, 
                                                        function_set=self.current_set.copy(), 
                                                        iterations=self.iterations)

        self.next_level = self.current_level + 1
        
        return self.current_max_value, self.floor, self.kept_value, self.next_level, self.current_set
        
    @staticmethod
    def _value_estimate(items, capacity, level, function_set, iterations):
        '''This finds the best fractional value that the bag can hold.'''
        
        # initialize. since this is an update, it is called when branches
        # "move left" - therefore, the current level will be 0 (item not included)
        estimated_value = 0
        if iterations > 0:
            function_set[level] = 0
            function_set[level+1:] = 1
        
        # for new branch, calculate potential/fractional value creation
        for i, item in enumerate(items):
            if (item.weight < capacity) & (function_set[i] == 1):
                estimated_value += item.value
                capacity -= item.weight
            elif function_set[i] == 0:
                pass
            else:
                return(estimated_value + capacity*item.density)

        return estimated_value
    
    @staticmethod
    def _update_floor_and_kept_value(items, capacity, level, function_set, iterations):
        '''This finds the floor and "starting value" for the node in the new branch.''' 
        
        # initialize. set later values to 0 - this allows us to loop through the 
        # branch fully without accounting for those items' value
        floor = capacity
        kept_value = 0
        if iterations > 0:
            function_set[level] = 0
            function_set[level+1:] = 0

        for i, item in enumerate(items):
            if (item.weight < capacity) & (function_set[i] == 1):
                floor -= item.weight
                kept_value += item.value
            elif function_set[i] == 0:
                pass

        return floor, kept_value
    
    def _load_data(self):
        '''Takes Coursera input files splits them, and creates a tuple in 
        their preferred format. Sorted - needs to be unsorted for output.'''
    
        def density_sort(item_list):
            return(sorted(item_list, key=lambda item:-item.density))

        Item = namedtuple("Item", ['index', 'value', 'weight', 'density'])

        # parse the input
        lines = self.input_data.split('\n')

        firstLine = lines[0].split()
        item_count = int(firstLine[0])
        capacity = int(firstLine[1])

        items = []

        for i in range(1, item_count+1):
            line = lines[i]
            parts = line.split()
            items.append(Item(i-1, int(parts[0]), int(parts[1]), 
                              float(parts[0])/float(parts[1])))

        items = density_sort(items)

        return items, item_count, capacity
    
    def _generate_output(self):
        '''Since we sorted the output to improve the runtime, we need to resort
           it back to the original value for grading.'''
        
        # sort "back" the selected items
        index_list = []
        for i, item in enumerate(self.items):
            index_list.append(self.items[i].index)
            
        zipped_list = zip(index_list, self.best_set)
        resorted_selection = sorted(list(zipped_list), key=lambda x: x[0])
        final_selection = [y for (x, y) in resorted_selection]
        
        # prepare the solution in the specified output format
        output_data = str(self.best_value) + ' ' + str(0) + '\n '
        output_data += ' '.join(map(str, final_selection))
        
        return output_data

In [21]:
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
testobj = dfs(input_data=input_data)
testobj.explore_branch()

# exhaust
while np.sum(testobj.current_set) > 0:
    testobj.explore_branch()

testobj._generate_output()

'99751 0\n 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0'

## Greedy